## Preprocessing

In [7]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [25]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(['EIN', 'NAME'], axis=1)

In [26]:
# Determine the number of unique values in each column.
unique_values = application_df.nunique()

In [27]:
# Look at APPLICATION_TYPE value counts for binning
application_df["APPLICATION_TYPE"].value_counts()

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [37]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
cutoff_value = 500
application_types_to_replace = unique_values[unique_values.index == 'APPLICATION_TYPE'][unique_values['APPLICATION_TYPE'] < cutoff_value].index.tolist()

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app, "Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()




KeyError: True

In [29]:
# Look at CLASSIFICATION value counts for binning
classification_counts = application_df['CLASSIFICATION'].value_counts()
print(classification_counts)

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64


In [30]:
# You may find it helpful to look at CLASSIFICATION value counts >1
classification_counts_greater_than_1 = classification_counts[classification_counts > 1]
print(classification_counts_greater_than_1)

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: CLASSIFICATION, dtype: int64


In [32]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
cutoff_values = 15
classifications_to_replace = classification_counts[classification_counts < cutoff_value].index.tolist()

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
Other     1484
C7000      777
Name: CLASSIFICATION, dtype: int64

In [33]:
# Convert categorical data to numeric with `pd.get_dummies`
application_df_encoded = pd.get_dummies(application_df)

In [34]:
# Split our preprocessed data into our features and target arrays
X = application_df_encoded.drop('IS_SUCCESSFUL', axis=1)
y = application_df_encoded['IS_SUCCESSFUL']

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

In [35]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [38]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

nn = tf.keras.models.Sequential()
input_features = X.shape[1]
hidden_layer1_nodes = 128
hidden_layer2_nodes = 60


# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_layer1_nodes, input_dim=input_features, activation='relu'))


# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_layer2_nodes, activation='relu'))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 128)               6784      
                                                                 
 dense_1 (Dense)             (None, 60)                7740      
                                                                 
 dense_2 (Dense)             (None, 1)                 61        
                                                                 
Total params: 14585 (56.97 KB)
Trainable params: 14585 (56.97 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [39]:
nn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [45]:
epochs = 100
history = nn.fit(X_train_scaled, y_train, epochs=epochs, shuffle=True, validation_data=(X_test_scaled, y_test))

Epoch 1/100
  1/804 [..............................] - ETA: 4s - loss: 0.5431 - accuracy: 0.7812

804/804 [==============================] - 2s 2ms/step - loss: 0.5323 - accuracy: 0.7425 - val_loss: 0.5665 - val_accuracy: 0.7256
Epoch 2/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5324 - accuracy: 0.7426 - val_loss: 0.5690 - val_accuracy: 0.7251
Epoch 3/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5316 - accuracy: 0.7424 - val_loss: 0.5687 - val_accuracy: 0.7249
Epoch 4/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5382 - accuracy: 0.7425 - val_loss: 0.5679 - val_accuracy: 0.7249
Epoch 5/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5318 - accuracy: 0.7415 - val_loss: 0.5700 - val_accuracy: 0.7237
Epoch 6/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5317 - accuracy: 0.7420 - val_loss: 0.5690 - val_accuracy: 0.7241
Epoch 7/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5320 - accuracy: 0.7427 - val_loss: 0.5685 - val_accuracy: 0.7255
Epoch 8/100

In [43]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5645 - accuracy: 0.7247 - 208ms/epoch - 776us/step
Loss: 0.5645366311073303, Accuracy: 0.7246647477149963


In [44]:
hdf5_filename = 'AlphabetSoupCharity.h5'

# Save the model to HDF5 file
nn.save(hdf5_filename)

c:\Users\desmo\anaconda3\envs\EdX\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
